<h1><center> Oversampling approach </center></h1>

## Introduction
__We will use some technique to extension the minority class (ADASYN)  and using some classification technique to classify this problem.__
__The evaluation result is include in this note. I  wish that the evaluation can convict this model.__
__library requires__

- pandas
- numpy
- sklearn
- imblearn
- catboost

In [7]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

## Data preparation

In [2]:
df = pd.read_csv('Some_Sampling.csv',index_col=None)
X = df[['X_AGE_MTH','FIRST_INSURE_AGE','X_ACTIVE_RIDER_CNT',
        'X_PAID_RIDER_ANP_AMT','STD_OCCUPATION_CD',
        'oldPremium','FIRST_LIFE_PRODUCT_RK',
        'X_PAID_POLICY_ANP_AMT',
        'X_RIDER_DISEASE_SUM_ASSURE_AMT',
        'X_POLICY_SUM_ASSURE_AMT',
        'X_RIDER_DISEASE_SUM_ASSURE_AMT',
        'X_BASEPLAN_TERM_ASSURE_AMT',
        'X_RIDER_SUM_ASSURE_AMT',
        'X_ACTIVE_POLICY_CNT',
        'X_COMPLAINT_L60M_CNT',
        'X_RIDER_PAYER_SUM_ASSURE_AMT',
        'ACTIVE_PA_INS_ACCTS_CNT',
        'X_CUSTOMER_STATUS',
        'X_COMP_POLICY_SRV_L60M_CNT',
        'X_POLICY_CONTACT_COMPLETE_CNT',
        'X_TLC_ACTIVE_FLG',
        'X_CUST_BENEFICIARY_MOTHER_FLG',
        'X_CUST_BENEFICIARY_CHILD_FLG',
        'VALID_EMAIL_FLG',
        'X_PAY_CHANNEL_BANK_FLG',
        'VALID_HOME_PHONE_FLG',
        'X_SHORT_PAY_FLG']]

Y = df[['buyFlag']]

df_test = pd.read_csv('GIO DATA FOR MINNING_clean_v0.1.csv',index_col=None)
X_valid = df_test[['X_AGE_MTH','FIRST_INSURE_AGE','X_ACTIVE_RIDER_CNT',
        'X_PAID_RIDER_ANP_AMT','STD_OCCUPATION_CD',
        'oldPremium','FIRST_LIFE_PRODUCT_RK',
        'X_PAID_POLICY_ANP_AMT',
        'X_RIDER_DISEASE_SUM_ASSURE_AMT',
        'X_POLICY_SUM_ASSURE_AMT',
        'X_RIDER_DISEASE_SUM_ASSURE_AMT',
        'X_BASEPLAN_TERM_ASSURE_AMT',
        'X_RIDER_SUM_ASSURE_AMT',
        'X_ACTIVE_POLICY_CNT',
        'X_COMPLAINT_L60M_CNT',
        'X_RIDER_PAYER_SUM_ASSURE_AMT',
        'ACTIVE_PA_INS_ACCTS_CNT',
        'X_CUSTOMER_STATUS',
        'X_COMP_POLICY_SRV_L60M_CNT',
        'X_POLICY_CONTACT_COMPLETE_CNT',
        'X_TLC_ACTIVE_FLG',
        'X_CUST_BENEFICIARY_MOTHER_FLG',
        'X_CUST_BENEFICIARY_CHILD_FLG',
        'VALID_EMAIL_FLG',
        'X_PAY_CHANNEL_BANK_FLG',
        'VALID_HOME_PHONE_FLG',
        'X_SHORT_PAY_FLG']]

Y_valid = df_test[['buyFlag']]

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 27 columns):
X_AGE_MTH                         3372 non-null float64
FIRST_INSURE_AGE                  3372 non-null float64
X_ACTIVE_RIDER_CNT                3372 non-null float64
X_PAID_RIDER_ANP_AMT              3372 non-null float64
STD_OCCUPATION_CD                 3372 non-null float64
oldPremium                        3372 non-null float64
FIRST_LIFE_PRODUCT_RK             3372 non-null float64
X_PAID_POLICY_ANP_AMT             3372 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    3372 non-null float64
X_POLICY_SUM_ASSURE_AMT           3372 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    3372 non-null float64
X_BASEPLAN_TERM_ASSURE_AMT        3372 non-null float64
X_RIDER_SUM_ASSURE_AMT            3372 non-null float64
X_ACTIVE_POLICY_CNT               3372 non-null float64
X_COMPLAINT_L60M_CNT              3372 non-null float64
X_RIDER_PAYER_SUM_ASSURE_AMT      3372 non-null float

In [4]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 1 columns):
buyFlag    3372 non-null int64
dtypes: int64(1)
memory usage: 26.4 KB


## Plain Text

In [9]:
# Use plain text for all the prediction set
ACI = ['ACTIVE', 'COMPLETE', 'INACTIVE']
YN = ['N', 'Y']
print(ACI)
print(YN)

['ACTIVE', 'COMPLETE', 'INACTIVE']
['N', 'Y']


In [5]:
X['X_CUSTOMER_STATUS'] = pd.Categorical(X['X_CUSTOMER_STATUS'], 
                categories=ACI).codes
X['X_TLC_ACTIVE_FLG'] = pd.Categorical(X['X_TLC_ACTIVE_FLG'], 
                categories=YN).codes
X['X_CUST_BENEFICIARY_MOTHER_FLG'] = pd.Categorical(X['X_CUST_BENEFICIARY_MOTHER_FLG'], 
                categories=YN).codes
X['X_CUST_BENEFICIARY_CHILD_FLG'] = pd.Categorical(X['X_CUST_BENEFICIARY_CHILD_FLG'], 
                categories=YN).codes
X['VALID_EMAIL_FLG'] = pd.Categorical(X['VALID_EMAIL_FLG'], 
                categories=YN).codes
X['X_PAY_CHANNEL_BANK_FLG'] = pd.Categorical(X['X_PAY_CHANNEL_BANK_FLG'], 
                categories=YN).codes
X['X_SHORT_PAY_FLG'] = pd.Categorical(X['X_SHORT_PAY_FLG'], 
                categories=YN).codes
X['VALID_HOME_PHONE_FLG'] = pd.Categorical(X['VALID_HOME_PHONE_FLG'], 
                categories=YN).codes

In [6]:
X_valid['X_CUSTOMER_STATUS'] = pd.Categorical(X_valid['X_CUSTOMER_STATUS'], 
                categories=ACI)).codes
X_valid['X_TLC_ACTIVE_FLG'] = pd.Categorical(X_valid['X_TLC_ACTIVE_FLG'], 
                categories=YN).codes
X_valid['X_CUST_BENEFICIARY_MOTHER_FLG'] = pd.Categorical(X_valid['X_CUST_BENEFICIARY_MOTHER_FLG'], 
                categories=YN).codes
X_valid['X_CUST_BENEFICIARY_CHILD_FLG'] = pd.Categorical(X_valid['X_CUST_BENEFICIARY_CHILD_FLG'], 
                categories=YN).codes
X_valid['VALID_EMAIL_FLG'] = pd.Categorical(X_valid['VALID_EMAIL_FLG'], 
                categories=YN).codes
X_valid['X_PAY_CHANNEL_BANK_FLG'] = pd.Categorical(X_valid['X_PAY_CHANNEL_BANK_FLG'], 
                categories=YN).codes
X_valid['X_SHORT_PAY_FLG'] = pd.Categorical(X_valid['X_SHORT_PAY_FLG'], 
                categories=YN).codes
X_valid['VALID_HOME_PHONE_FLG'] = pd.Categorical(X_valid['VALID_HOME_PHONE_FLG'], 
                categories=YN).codes

## Train and Test split

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0, )

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3372 entries, 2615 to 1563
Data columns (total 27 columns):
X_AGE_MTH                         3372 non-null float64
FIRST_INSURE_AGE                  3372 non-null float64
X_ACTIVE_RIDER_CNT                3372 non-null float64
X_PAID_RIDER_ANP_AMT              3372 non-null float64
STD_OCCUPATION_CD                 3372 non-null float64
oldPremium                        3372 non-null float64
FIRST_LIFE_PRODUCT_RK             3372 non-null float64
X_PAID_POLICY_ANP_AMT             3372 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    3372 non-null float64
X_POLICY_SUM_ASSURE_AMT           3372 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    3372 non-null float64
X_BASEPLAN_TERM_ASSURE_AMT        3372 non-null float64
X_RIDER_SUM_ASSURE_AMT            3372 non-null float64
X_ACTIVE_POLICY_CNT               3372 non-null float64
X_COMPLAINT_L60M_CNT              3372 non-null float64
X_RIDER_PAYER_SUM_ASSURE_AMT      3372 non-null fl

In [9]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 27 columns):
X_AGE_MTH                         0 non-null float64
FIRST_INSURE_AGE                  0 non-null float64
X_ACTIVE_RIDER_CNT                0 non-null float64
X_PAID_RIDER_ANP_AMT              0 non-null float64
STD_OCCUPATION_CD                 0 non-null float64
oldPremium                        0 non-null float64
FIRST_LIFE_PRODUCT_RK             0 non-null float64
X_PAID_POLICY_ANP_AMT             0 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    0 non-null float64
X_POLICY_SUM_ASSURE_AMT           0 non-null float64
X_RIDER_DISEASE_SUM_ASSURE_AMT    0 non-null float64
X_BASEPLAN_TERM_ASSURE_AMT        0 non-null float64
X_RIDER_SUM_ASSURE_AMT            0 non-null float64
X_ACTIVE_POLICY_CNT               0 non-null float64
X_COMPLAINT_L60M_CNT              0 non-null float64
X_RIDER_PAYER_SUM_ASSURE_AMT      0 non-null float64
ACTIVE_PA_INS_ACCTS_CNT           0 non-null float64
X_CUST

# Oversampling by SMOTE 

In [10]:
from imblearn.over_sampling import ADASYN
sm = ADASYN(sampling_strategy='minority')
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

G:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Experiment

In [21]:
from catboost import CatBoostClassifier
clf_best = CatBoostClassifier(depth = 10, iterations = 50000,  l2_leaf_reg = 9, learning_rate = 0.15) # best parameter: depth = 10, iterations = 50000,  l2_leaf_reg = 9, learning_rate = 0.15
clf_best.fit(X_train_res, y_train_res)

y_predict_train = clf_best.predict(X_train)

y_predict_test = clf_best.predict(X_test)

0:	learn: 0.6397072	total: 905ms	remaining: 12h 34m 15s
1:	learn: 0.6086225	total: 1.54s	remaining: 10h 43m 24s
2:	learn: 0.5812264	total: 2.19s	remaining: 10h 8m 54s
3:	learn: 0.5553151	total: 2.76s	remaining: 9h 34m 52s
4:	learn: 0.5329010	total: 3.33s	remaining: 9h 15m 19s
5:	learn: 0.5208434	total: 3.59s	remaining: 8h 18m 59s
6:	learn: 0.5094703	total: 4.18s	remaining: 8h 18m 7s
7:	learn: 0.4985137	total: 4.81s	remaining: 8h 21m 5s
8:	learn: 0.4893003	total: 5.43s	remaining: 8h 22m 48s
9:	learn: 0.4832800	total: 5.97s	remaining: 8h 17m 47s
10:	learn: 0.4778756	total: 6.51s	remaining: 8h 12m 50s
11:	learn: 0.4739490	total: 7.05s	remaining: 8h 9m 14s
12:	learn: 0.4664126	total: 7.58s	remaining: 8h 6m 6s
13:	learn: 0.4645425	total: 7.66s	remaining: 7h 35m 49s
14:	learn: 0.4603061	total: 8.25s	remaining: 7h 38m 6s
15:	learn: 0.4575560	total: 8.9s	remaining: 7h 43m 15s
16:	learn: 0.4529649	total: 9.52s	remaining: 7h 46m 42s
17:	learn: 0.4493579	total: 10.1s	remaining: 7h 46m 31s
18:	lea

147:	learn: 0.2890599	total: 1m 19s	remaining: 7h 26m 29s
148:	learn: 0.2886560	total: 1m 20s	remaining: 7h 26m 30s
149:	learn: 0.2878226	total: 1m 20s	remaining: 7h 26m 32s
150:	learn: 0.2868505	total: 1m 21s	remaining: 7h 26m 58s
151:	learn: 0.2864399	total: 1m 21s	remaining: 7h 27m 8s
152:	learn: 0.2860471	total: 1m 22s	remaining: 7h 27m 6s
153:	learn: 0.2851705	total: 1m 22s	remaining: 7h 26m 59s
154:	learn: 0.2851460	total: 1m 23s	remaining: 7h 26m 57s
155:	learn: 0.2846561	total: 1m 23s	remaining: 7h 26m 56s
156:	learn: 0.2845722	total: 1m 24s	remaining: 7h 26m 54s
157:	learn: 0.2838871	total: 1m 24s	remaining: 7h 26m 49s
158:	learn: 0.2838336	total: 1m 25s	remaining: 7h 26m 46s
159:	learn: 0.2828648	total: 1m 26s	remaining: 7h 26m 44s
160:	learn: 0.2822760	total: 1m 26s	remaining: 7h 26m 44s
161:	learn: 0.2812606	total: 1m 27s	remaining: 7h 26m 37s
162:	learn: 0.2807323	total: 1m 27s	remaining: 7h 26m 34s
163:	learn: 0.2799312	total: 1m 28s	remaining: 7h 26m 28s
164:	learn: 0.27

291:	learn: 0.2368564	total: 2m 36s	remaining: 7h 22m 37s
292:	learn: 0.2368486	total: 2m 36s	remaining: 7h 22m 34s
293:	learn: 0.2364570	total: 2m 37s	remaining: 7h 22m 34s
294:	learn: 0.2363561	total: 2m 37s	remaining: 7h 22m 35s
295:	learn: 0.2361258	total: 2m 38s	remaining: 7h 22m 34s
296:	learn: 0.2360988	total: 2m 38s	remaining: 7h 22m 34s
297:	learn: 0.2357037	total: 2m 39s	remaining: 7h 22m 44s
298:	learn: 0.2356521	total: 2m 39s	remaining: 7h 22m 45s
299:	learn: 0.2355549	total: 2m 40s	remaining: 7h 22m 43s
300:	learn: 0.2349477	total: 2m 40s	remaining: 7h 22m 43s
301:	learn: 0.2348036	total: 2m 41s	remaining: 7h 22m 41s
302:	learn: 0.2346670	total: 2m 41s	remaining: 7h 22m 40s
303:	learn: 0.2340551	total: 2m 42s	remaining: 7h 22m 38s
304:	learn: 0.2339276	total: 2m 42s	remaining: 7h 22m 36s
305:	learn: 0.2337966	total: 2m 43s	remaining: 7h 22m 34s
306:	learn: 0.2336669	total: 2m 44s	remaining: 7h 22m 32s
307:	learn: 0.2336214	total: 2m 44s	remaining: 7h 22m 28s
308:	learn: 0.

433:	learn: 0.2064035	total: 3m 51s	remaining: 7h 21m 35s
434:	learn: 0.2063203	total: 3m 52s	remaining: 7h 21m 35s
435:	learn: 0.2063151	total: 3m 53s	remaining: 7h 21m 40s
436:	learn: 0.2062313	total: 3m 53s	remaining: 7h 21m 48s
437:	learn: 0.2061688	total: 3m 54s	remaining: 7h 21m 54s
438:	learn: 0.2058880	total: 3m 54s	remaining: 7h 22m
439:	learn: 0.2055694	total: 3m 55s	remaining: 7h 22m 6s
440:	learn: 0.2053583	total: 3m 56s	remaining: 7h 22m 12s
441:	learn: 0.2052893	total: 3m 56s	remaining: 7h 22m 20s
442:	learn: 0.2051486	total: 3m 57s	remaining: 7h 22m 26s
443:	learn: 0.2048292	total: 3m 57s	remaining: 7h 22m 33s
444:	learn: 0.2048168	total: 3m 58s	remaining: 7h 22m 41s
445:	learn: 0.2034779	total: 3m 59s	remaining: 7h 22m 39s
446:	learn: 0.2034763	total: 3m 59s	remaining: 7h 22m 36s
447:	learn: 0.2034619	total: 4m	remaining: 7h 22m 33s
448:	learn: 0.2033075	total: 4m	remaining: 7h 22m 31s
449:	learn: 0.2032687	total: 4m 1s	remaining: 7h 22m 30s
450:	learn: 0.2030556	total:

576:	learn: 0.1880785	total: 5m 8s	remaining: 7h 20m 29s
577:	learn: 0.1880741	total: 5m 9s	remaining: 7h 20m 27s
578:	learn: 0.1876529	total: 5m 9s	remaining: 7h 20m 26s
579:	learn: 0.1876523	total: 5m 10s	remaining: 7h 20m 18s
580:	learn: 0.1875452	total: 5m 10s	remaining: 7h 20m 18s
581:	learn: 0.1875193	total: 5m 11s	remaining: 7h 20m 16s
582:	learn: 0.1875148	total: 5m 11s	remaining: 7h 20m 15s
583:	learn: 0.1873552	total: 5m 12s	remaining: 7h 20m 16s
584:	learn: 0.1873531	total: 5m 12s	remaining: 7h 20m 18s
585:	learn: 0.1873514	total: 5m 13s	remaining: 7h 20m 20s
586:	learn: 0.1871781	total: 5m 13s	remaining: 7h 20m 22s
587:	learn: 0.1871739	total: 5m 14s	remaining: 7h 20m 25s
588:	learn: 0.1870313	total: 5m 15s	remaining: 7h 20m 28s
589:	learn: 0.1870119	total: 5m 15s	remaining: 7h 20m 34s
590:	learn: 0.1869786	total: 5m 16s	remaining: 7h 20m 39s
591:	learn: 0.1869674	total: 5m 16s	remaining: 7h 20m 46s
592:	learn: 0.1869542	total: 5m 17s	remaining: 7h 20m 53s
593:	learn: 0.186

...

49382:	learn: 0.0185989	total: 7h 7m 43s	remaining: 5m 20s
49383:	learn: 0.0185989	total: 7h 7m 44s	remaining: 5m 20s
49384:	learn: 0.0185989	total: 7h 7m 44s	remaining: 5m 19s
49385:	learn: 0.0185989	total: 7h 7m 45s	remaining: 5m 19s
49386:	learn: 0.0185989	total: 7h 7m 46s	remaining: 5m 18s
49387:	learn: 0.0185989	total: 7h 7m 46s	remaining: 5m 18s
49388:	learn: 0.0185989	total: 7h 7m 47s	remaining: 5m 17s
49389:	learn: 0.0185989	total: 7h 7m 47s	remaining: 5m 17s
49390:	learn: 0.0185989	total: 7h 7m 48s	remaining: 5m 16s
49391:	learn: 0.0185989	total: 7h 7m 48s	remaining: 5m 15s
49392:	learn: 0.0185988	total: 7h 7m 49s	remaining: 5m 15s
49393:	learn: 0.0185988	total: 7h 7m 49s	remaining: 5m 14s
49394:	learn: 0.0185988	total: 7h 7m 50s	remaining: 5m 14s
49395:	learn: 0.0185987	total: 7h 7m 50s	remaining: 5m 13s
49396:	learn: 0.0185987	total: 7h 7m 51s	remaining: 5m 13s
49397:	learn: 0.0185984	total: 7h 7m 51s	remaining: 5m 12s
49398:	learn: 0.0185984	total: 7h 7m 52s	remaining: 5m 1

49522:	learn: 0.0185427	total: 7h 8m 57s	remaining: 4m 7s
49523:	learn: 0.0185427	total: 7h 8m 57s	remaining: 4m 7s
49524:	learn: 0.0185427	total: 7h 8m 58s	remaining: 4m 6s
49525:	learn: 0.0185426	total: 7h 8m 58s	remaining: 4m 6s
49526:	learn: 0.0185426	total: 7h 8m 59s	remaining: 4m 5s
49527:	learn: 0.0185426	total: 7h 9m	remaining: 4m 5s
49528:	learn: 0.0185426	total: 7h 9m	remaining: 4m 4s
49529:	learn: 0.0185426	total: 7h 9m	remaining: 4m 4s
49530:	learn: 0.0185426	total: 7h 9m 1s	remaining: 4m 3s
49531:	learn: 0.0185426	total: 7h 9m 1s	remaining: 4m 3s
49532:	learn: 0.0185426	total: 7h 9m 2s	remaining: 4m 2s
49533:	learn: 0.0185426	total: 7h 9m 3s	remaining: 4m 2s
49534:	learn: 0.0185426	total: 7h 9m 3s	remaining: 4m 1s
49535:	learn: 0.0185424	total: 7h 9m 4s	remaining: 4m 1s
49536:	learn: 0.0185424	total: 7h 9m 4s	remaining: 4m
49537:	learn: 0.0185423	total: 7h 9m 5s	remaining: 4m
49538:	learn: 0.0185422	total: 7h 9m 5s	remaining: 3m 59s
49539:	learn: 0.0185422	total: 7h 9m 6s	

49663:	learn: 0.0185388	total: 7h 10m 10s	remaining: 2m 54s
49664:	learn: 0.0185388	total: 7h 10m 10s	remaining: 2m 54s
49665:	learn: 0.0185388	total: 7h 10m 11s	remaining: 2m 53s
49666:	learn: 0.0185388	total: 7h 10m 11s	remaining: 2m 53s
49667:	learn: 0.0185388	total: 7h 10m 12s	remaining: 2m 52s
49668:	learn: 0.0185358	total: 7h 10m 13s	remaining: 2m 52s
49669:	learn: 0.0185358	total: 7h 10m 13s	remaining: 2m 51s
49670:	learn: 0.0185357	total: 7h 10m 14s	remaining: 2m 50s
49671:	learn: 0.0185357	total: 7h 10m 14s	remaining: 2m 50s
49672:	learn: 0.0185357	total: 7h 10m 15s	remaining: 2m 49s
49673:	learn: 0.0185357	total: 7h 10m 15s	remaining: 2m 49s
49674:	learn: 0.0185357	total: 7h 10m 16s	remaining: 2m 48s
49675:	learn: 0.0185357	total: 7h 10m 16s	remaining: 2m 48s
49676:	learn: 0.0185357	total: 7h 10m 17s	remaining: 2m 47s
49677:	learn: 0.0185357	total: 7h 10m 17s	remaining: 2m 47s
49678:	learn: 0.0185357	total: 7h 10m 17s	remaining: 2m 46s
49679:	learn: 0.0185357	total: 7h 10m 18

49801:	learn: 0.0185325	total: 7h 11m 16s	remaining: 1m 42s
49802:	learn: 0.0185324	total: 7h 11m 16s	remaining: 1m 42s
49803:	learn: 0.0185324	total: 7h 11m 17s	remaining: 1m 41s
49804:	learn: 0.0185324	total: 7h 11m 17s	remaining: 1m 41s
49805:	learn: 0.0185324	total: 7h 11m 18s	remaining: 1m 40s
49806:	learn: 0.0185324	total: 7h 11m 18s	remaining: 1m 40s
49807:	learn: 0.0185324	total: 7h 11m 19s	remaining: 1m 39s
49808:	learn: 0.0185324	total: 7h 11m 19s	remaining: 1m 39s
49809:	learn: 0.0185324	total: 7h 11m 20s	remaining: 1m 38s
49810:	learn: 0.0185324	total: 7h 11m 20s	remaining: 1m 38s
49811:	learn: 0.0185322	total: 7h 11m 21s	remaining: 1m 37s
49812:	learn: 0.0185322	total: 7h 11m 21s	remaining: 1m 37s
49813:	learn: 0.0185321	total: 7h 11m 22s	remaining: 1m 36s
49814:	learn: 0.0185321	total: 7h 11m 22s	remaining: 1m 36s
49815:	learn: 0.0185321	total: 7h 11m 23s	remaining: 1m 35s
49816:	learn: 0.0185321	total: 7h 11m 23s	remaining: 1m 35s
49817:	learn: 0.0185321	total: 7h 11m 24

49940:	learn: 0.0184629	total: 7h 12m 25s	remaining: 30.7s
49941:	learn: 0.0184628	total: 7h 12m 26s	remaining: 30.1s
49942:	learn: 0.0184625	total: 7h 12m 26s	remaining: 29.6s
49943:	learn: 0.0184618	total: 7h 12m 27s	remaining: 29.1s
49944:	learn: 0.0184617	total: 7h 12m 27s	remaining: 28.6s
49945:	learn: 0.0184617	total: 7h 12m 28s	remaining: 28.1s
49946:	learn: 0.0184617	total: 7h 12m 28s	remaining: 27.5s
49947:	learn: 0.0184617	total: 7h 12m 29s	remaining: 27s
49948:	learn: 0.0184616	total: 7h 12m 29s	remaining: 26.5s
49949:	learn: 0.0184616	total: 7h 12m 30s	remaining: 26s
49950:	learn: 0.0184616	total: 7h 12m 30s	remaining: 25.5s
49951:	learn: 0.0184616	total: 7h 12m 31s	remaining: 24.9s
49952:	learn: 0.0184616	total: 7h 12m 31s	remaining: 24.4s
49953:	learn: 0.0184616	total: 7h 12m 32s	remaining: 23.9s
49954:	learn: 0.0184615	total: 7h 12m 33s	remaining: 23.4s
49955:	learn: 0.0184615	total: 7h 12m 33s	remaining: 22.9s
49956:	learn: 0.0184612	total: 7h 12m 34s	remaining: 22.3s
4

# Training

In [27]:
from sklearn.metrics import accuracy_score
print(
    "The Accuracy of Catboost + ADSYN + top 28 feature with repect to the train set"
)
print("::", accuracy_score(y_train, y_predict_train) * 100)

The Accuracy of Catboost + ADSYN + top 28 feature with repect to the train set
:: 99.85172004744959


In [28]:
from sklearn.metrics import confusion_matrix
print("The Confusion Metrix of Catboost + ADSYN + top 28 feature of train set")
unique_label = np.unique(y_train)
print(pd.DataFrame(confusion_matrix(y_train,
                                    y_predict_train,labels=unique_label), 
                index=['true:{:}'.format(x) for x in unique_label], 
                columns=['pred:{:}'.format(x) for x in unique_label]))

The Confusion Metrix of Catboost + ADSYN + top 28 feature of train set
        pred:0  pred:1
true:0    1998       2
true:1       3    1369


# Test

In [29]:
from sklearn.metrics import f1_score
#predict test set
y_predict_valid = clf_best.predict(X_valid)
print(
    "The Accuracy of Catboostt + ADSYN + top 27 feature with repect to the valid set"
)
print("::", accuracy_score(Y_valid, y_predict_valid) * 100)

print("the F1 score of Catboost + ADSYN + top 27 feature with repect to the valid set::"
)
print("::",f1_score(Y_valid, y_predict_valid))

The Accuracy of Catboostt + ADSYN + top 27 feature with repect to the valid set
:: 78.60273203954878
the F1 score of Catboost + ADSYN + top 27 feature with repect to the valid set::
:: 0.057446037800174723


In [30]:
print("The Confusion Metrix of Catboost + ADSYN + top 27 feature of valid set")
unique_label = np.unique(Y_valid)
print(pd.DataFrame(confusion_matrix(Y_valid,
                                    y_predict_valid,labels=unique_label), 
                index=['true:{:}'.format(x) for x in unique_label], 
                columns=['pred:{:}'.format(x) for x in unique_label]))

The Confusion Metrix of Catboost + ADSYN + top 27 feature of valid set
        pred:0  pred:1
true:0  161149   44211
true:1      24    1348


In [26]:
clf_best.save_model("bestmodel_1")